http://alistapart.com/article/getoutbindingsituations

In [1]:
var alert = function(str) {
    console.log(str);
}

In [2]:

var john = {
  name: 'John',
  greet: function(person) {
    alert("Hi " + person + ", my name is " + name);
  }
};
john.greet("Mark");

In [3]:
name = 'Ray'; // Or explictly: window.name = 'Ray';
var john = {
  name: 'John',
  greet: function(person) {
    alert("Hi " + person + ", my name is " + name);
  }
};
john.greet("Mark");

In [4]:
name = 'Ray';
var john = {
  name: 'John',
  greet: function(person) {
    alert("Hi " + person + ", my name is " + this.name);
  }
};
john.greet("Mark");
var fx = john.greet;
fx("Mark");

### RECOGNIZING BINDING-SENSITIVE CODE PATTERNS

In [5]:
var alert = function(str) {
    console.log(str);
}

function Person(first, last, age) {
  this.first = first;
  this.last = last;
  this.age = age;
}
Person.prototype = {
  getFullName: function() {
    alert(this.first + ' ' + this.last);
  },
  greet: function(other) {
    alert("Hi " + other.first + ", I'm " + this.first + ".");
  }
};

In [6]:
var elodie = new Person('Elodie', 'Jaubert', 27);
var christophe = new Person('Christophe', 'Porteneuve', 30);
christophe.greet(elodie);

In [7]:
function times(n, fx, arg) {
  for (var index = 0; index < n; ++index) {
    fx(arg);
  }
}
times(3, christophe.greet, elodie);
times(1, elodie.getFullName);

### APPLY WITHIN

In [8]:
var fx = christophe.greet;
fx.apply(christophe, [elodie]);

Hi Elodie, I'm Christophe.


### CALL NOW

In [9]:
var fx = christophe.greet;
fx.call(christophe, elodie);

Hi Elodie, I'm Christophe.


In [10]:
function createBoundedWrapper(object, method) {
  return function() {
    return method.apply(object, arguments);
  };
}
var chrisGreet = createBoundedWrapper(christophe, christophe.greet);
chrisGreet(elodie);

Hi Elodie, I'm Christophe.


### JavaScript frameworks do it
Our createBoundedWrapper function is neat, but may prove a bit unwieldy.

In [11]:
var chrisGreet = christophe.greet.bind(christophe);
chrisGreet(elodie);

Hi Elodie, I'm Christophe.


In [12]:
var coolBehavior = {
  // ...
  toggle: function(enabled) {
    this.enabled = enabled;
    // ...
  },
  // ...
};
coolBehavior.enable = coolBehavior.toggle.bind(coolBehavior, true);
coolBehavior.disable = coolBehavior.toggle.bind(coolBehavior, false);

alert(coolBehavior)

coolBehavior.enable();
alert(coolBehavior)

coolBehavior.disable();
alert(coolBehavior)

{}
{"enabled":true}
{"enabled":false}


In [13]:
function times (count, fx) {
  for (var index = 0; index < count; ++index) {
    fx();
  }
}
var threeTimes = times.bind(null, 3);
threeTimes(function() {alert('abc')});

abc
abc
abc


So as a side note, with Prototype 1.6, if you’re only interested in pre-filling, prefer curry—it preserves the current binding and focuses on argument pre-filling

var threeTimes = times.curry(3);

### Should you even bind?
binding is overkill.  Specifically, there’s a code pattern in which binding can be replaced, with significant performance profit, by using the lexical closure

In [14]:
var some = {
  // ...
  processItems: function() {
    var that = this;
    this.items.each(function(item) {
      // Process item
      that.markItemAsProcessed(item);
    });
  },
  // ...
};

//some.processItems();

### Takeaway points
- Any member access must be qualified with the object it pertains to, even when it is this.
- Any sort of function reference (assigning as a value, passing as an argument) loses the function’s original binding.
- JavaScript provides two equivalent ways of explicitly specifying a function’s binding when calling it: apply and call.
- Creating a “bound method reference” requires an anonymous wrapper function, and a calling cost. In specific situations, leveraging closures may be a better alternative.